In [1]:
from soirbhiochas import staidreamh

To address perceived minor issues, this actually changes some of the regex matching in Opers.Slenderize - marked with a v2 flag.


In [2]:
import importlib
importlib.reload(staidreamh)

<module 'soirbhiochas.staidreamh' from '/home/philtweir/Pearsanta/Cód/Gaeilge/soirbhíochas/soirbhiochas/staidreamh.py'>

In [3]:
from soirbhiochas.díolaim import Díolaim, CorrectionDict
from soirbhiochas.parsáil import Lexicon

In [4]:
CONLLU = "ga_idt-ud-train.conllu"
TYPOS = "suspected_typos.txt"

In [5]:
corrections: CorrectionDict = {}

with open(TYPOS, 'r') as f:
    correction_lines = f.readlines()
for line in correction_lines:
    from_form, from_upos, to_form, to_upos = line.split(',')
    corrections[(from_upos, from_form)] = (to_upos, to_form)

In [6]:
from soirbhiochas.leabharlann import CAOL_LE_CAOL

In [7]:
staidreamh.add_loadable("prefixes", "../wiktionary/wikt-irish-prefixes.txt")

lexicon = Lexicon()
lexicon.load()
staidreamh.add_loadable("lexicon", lexicon)

corpas = Díolaim.cruthaíodh_as_comhad(CONLLU, lexicon.find_by_token, corrections=corrections)
CAOL_LE_CAOL.set_sample_size(5)
counter, statistics = staidreamh.count_rule_by_word(CAOL_LE_CAOL, corpas.de_réir_focal())
print(f"Found {len(counter)} exceptions")
most_common = counter.most_common(150)
for row in zip(*[most_common[i::3] for i in range(3)]):
    row = [r[0] if r[1] == 1 else f"{r[0]} {r[1]}" for r in row]
    print(f"{row[0]: >20} {row[1]: >20} {row[2]: >20}")

counts = CAOL_LE_CAOL.get_counts()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95881/95881 [00:28<00:00, 3338.71it/s]

Found 144 exceptions
   t-uisceshaothrú 3            shaolré 2               insa 2
          istoíche 2       dlúthdhiosca 2        tráthrialta 2
        ghéarchéim 2        breisluacha 2            neamhní 2
        Céadchéime 2    ranníocaíochtaí 2      uisceshaothrú 2
      fhadtéarmach 2          eischósta 2            abalone 2
                biro                agena        Bordcheantair
       Bhordcheantar             tsecoind              magenta
          tonnchrith  Comptroller-General    pobalbhreitheanna
fadbhreathnaitheacha              twaddle         cillscannáin
           suimiúila            phíchairt               LINGUA
          lánchreach luath-athbheochantóirí     lúthchleasaíocht
            ghiorria            bánéadach                Dhera
     meitéareolaithe             pubcheol       consighneachta
       dtoghcheantar         dlíthairgthe             déanfidh
           dheonfidh              ndintar               dintar
           shaorfidh      gairmo

In [9]:
counts

{'self': <soirbhiochas.leabharlann.CaolLeCaol at 0x7f5ab815e490>,
 'count': {'repeated': {'all': {'pass': 90497,
    'fail': 160,
    'tick': 88892,
    'excepted': 1605},
   'only': {'pass': 90497, 'fail': 160, 'tick': 88892, 'excepted': 1605}},
  'by_lemma': {'all': {'pass': 6050,
    'fail': 113,
    'tick': 5644,
    'excepted': 406},
   'only': {'pass': 6050, 'fail': 113, 'tick': 5644, 'excepted': 406}},
  'by_form': {'all': {'pass': 11018,
    'fail': 144,
    'tick': 10471,
    'excepted': 547},
   'only': {'pass': 11018, 'fail': 144, 'tick': 10471, 'excepted': 547}},
  'by_demut': {'all': {'pass': 17973,
    'fail': 152,
    'tick': 16939,
    'excepted': 1034},
   'only': {'pass': 17973, 'fail': 152, 'tick': 16939, 'excepted': 1034}}},
 'exceptions': [{'self': <soirbhiochas.rialacha.RiailCumaisc at 0x7f5ab815e4f0>,
   'count': {'repeated': {'all': {'pass': 90,
      'fail': 90567,
      'tick': 90,
      'excepted': 0},
     'only': {'pass': 90, 'fail': 1675, 'tick': 90, 'exce

In [305]:
import pandas as pd
total_exceptions = counts['count']['repeated']['only']['fail'] + counts['count']['repeated']['only']['excepted']
exceptions = pd.DataFrame([
        dict(
            {k: ex['count']['repeated']['only'][k] for k in ('pass', 'tick', 'excepted', 'fail')},
            name=ex_na,
            samples=ex['samples']['tick'],
            samples_fail=ex['samples']['fail'],
        )
        for ex, ex_na in zip(counts['exceptions'], counts['exception_names'])
])
exceptions['total'] = total_exceptions
exceptions

,pass,tick,excepted,fail,name,samples,samples_fail,total
0,90,90,0,1675,...ach amháin 'ansin' agus 'anseo',"[Ansin, Anseo]","[gCeanncheathrú, faoina, neamhchead, ina, thra...",1765
1,893,893,0,782,...agus roinnt focail chumaisc,"[gCeanncheathrú, neamhchead, inrátaithe, comhi...","[faoina, ina, thraenacha, arís, lena]",1765
2,532,532,0,250,...agus roinnt reamhfhocail,"[faoina, ina, lena, inar, Lena]","[thraenacha, arís, biro, age, agena]",1765
3,39,39,0,211,...agus nuair a bhíonn 'ae' leathan i ndáirire,"[thraenacha, Gaelscoileanna, haturnaetha, Ghae...","[arís, biro, age, agena, Bordcheantair]",1765
4,48,48,0,163,...agus roinnt dobhríathair a thosaíonn le 'a',"[arís, age, n-atriam, ariamh, aréir]","[biro, agena, Bordcheantair, Bhordcheantar, ts...",1765
5,3,3,0,160,...agus 'féadfidh',[féadfidh],"[biro, agena, Bordcheantair, Bhordcheantar, ts...",1765


In [379]:
exceptions['offset'] = exceptions['tick'].cumsum() - exceptions['tick']

exceptions_by_level = pd.concat({
    0: pd.DataFrame([{
        'pass': counts['count']['repeated']['only']['pass'],
        'tick': counts['count']['repeated']['only']['tick'],
        'fail': counts['count']['repeated']['only']['fail'],
        'samples': counts['samples']['tick'],
        'samples_fail': counts['samples']['fail'],
        'offset': 0,
        'excepted': counts['count']['repeated']['only']['excepted'],
        'name': 'Caol le caol, leathan le leathan',
        'total': counts['count']['repeated']['only']['pass'] + counts['count']['repeated']['only']['fail']
    }]),
    1: exceptions
})

exceptions_by_level["final"] = False
for n, rows in exceptions_by_level.groupby(level=0):
    m = rows.index.get_level_values(level=1)[-1]
    exceptions_by_level.loc[(n, m), "final"] = True
exceptions_by_level

pass   tick  fail                                            samples  \
0 0  90497  88892   160                        [As, lár, na, tubaiste, is]   
1 0     90     90  1675                                     [Ansin, Anseo]   
  1    893    893   782  [gCeanncheathrú, neamhchead, inrátaithe, comhi...   
  2    532    532   250                    [faoina, ina, lena, inar, Lena]   
  3     39     39   211  [thraenacha, Gaelscoileanna, haturnaetha, Ghae...   
  4     48     48   163               [arís, age, n-atriam, ariamh, aréir]   
  5      3      3   160                                         [féadfidh]   

                                          samples_fail  offset  excepted  \
0 0  [biro, agena, Bordcheantair, Bhordcheantar, ts...       0      1605   
1 0  [gCeanncheathrú, faoina, neamhchead, ina, thra...       0         0   
  1              [faoina, ina, thraenacha, arís, lena]      90         0   
  2               [thraenacha, arís, biro, age, agena]     983         0   
  3            [arís, biro, age, agena, Bordcheantair]    1515         0   
  4  [biro, agena, Bordcheantair, Bhordcheantar, ts...    1554         0   
  5  [biro, agena, Bordcheantair, Bhordcheantar, ts...    1602         0   

                                               name  total  final  
0 0                Caol le caol, leathan le leathan  90657   True  
1 0              ...ach amháin 'ansin' agus 'anseo'   1765  False  
  1                  ...agus roinnt focail chumaisc   1765  False  
  2                     ...agus roinnt reamhfhocail   1765  False  
  3  ...agus nuair a bhíonn 'ae' leathan i ndáirire   1765  False  
  4  ...agus roinnt dobhríathair a thosaíonn le 'a'   1765  False  
  5                              ...agus 'féadfidh'   1765   True

In [380]:
exceptions_by_level = exceptions_by_level.reset_index()
exceptions_by_level

,level_0,level_1,pass,tick,fail,samples,samples_fail,offset,excepted,name,total,final
0,0,0,90497,88892,160,"[As, lár, na, tubaiste, is]","[biro, agena, Bordcheantair, Bhordcheantar, ts...",0,1605,"Caol le caol, leathan le leathan",90657,True
1,1,0,90,90,1675,"[Ansin, Anseo]","[gCeanncheathrú, faoina, neamhchead, ina, thra...",0,0,...ach amháin 'ansin' agus 'anseo',1765,False
2,1,1,893,893,782,"[gCeanncheathrú, neamhchead, inrátaithe, comhi...","[faoina, ina, thraenacha, arís, lena]",90,0,...agus roinnt focail chumaisc,1765,False
3,1,2,532,532,250,"[faoina, ina, lena, inar, Lena]","[thraenacha, arís, biro, age, agena]",983,0,...agus roinnt reamhfhocail,1765,False
4,1,3,39,39,211,"[thraenacha, Gaelscoileanna, haturnaetha, Ghae...","[arís, biro, age, agena, Bordcheantair]",1515,0,...agus nuair a bhíonn 'ae' leathan i ndáirire,1765,False
5,1,4,48,48,163,"[arís, age, n-atriam, ariamh, aréir]","[biro, agena, Bordcheantair, Bhordcheantar, ts...",1554,0,...agus roinnt dobhríathair a thosaíonn le 'a',1765,False
6,1,5,3,3,160,[féadfidh],"[biro, agena, Bordcheantair, Bhordcheantar, ts...",1602,0,...agus 'féadfidh',1765,True


In [307]:
#exceptionsp = pd.DataFrame(exceptions)
#exceptionsp['offset'] = exceptionsp['pass'].cumsum() - exceptionsp['pass']
#print(exceptionsp)
#exceptionsp['ix'] = exceptionsp.index
split = ('offset', 'pass', 'fail')
#exceptionsp = exceptionsp.melt(id_vars=['name', 'ix'], value_vars=split)
#for n, s in enumerate(split):
#    exceptionsp.loc[exceptionsp['variable'] == s, 'split'] = n

In [401]:
color_map_all = {
    'offset': '#EEEEEE',
    'pass': '#888888',
    'tick': '#888888',
    'fail': '#AAAAAA'
}
color_map_one = {
    'pass': '#27a827',
    'tick': '#27a827',
    'excepted': '#008080',
    'fail': '#008080'
}
#exceptions['cm_all'] = [color_map_all.get(k) for k in exceptionsp['variable']]
#exceptions['cm_one'] = [color_map_one.get(k) for k in exceptionsp['variable']]

In [504]:
import altair as alt
from vega_datasets import data

step = 20
overlap = 1
indent = 20

text = alt.Chart().mark_text(
    xOffset=alt.ExprRef(f'datum.level_0 * {indent}'),
    align='left',
    dx=-200,
    fontSize=alt.ExprRef('(1 - 0.2 * datum.level_0) * 20'),
    yOffset=-20
).encode(
    text='name'
)
split = ['offset', 'tick', 'excepted', 'fail']
all_split = alt.Chart().transform_fold(
    split
).transform_calculate(
    split=f'indexof({split}, datum.key)',
    color_key=f'datum.final && datum.key == "fail" ? "fail_all" : datum.key'
).mark_bar(
    xOffset=alt.ExprRef(f'datum.level_0 * {indent}'),
    yOffset=-10,
    height=6
).encode(
    x=alt.X('value:Q', stack="normalize", axis=alt.Axis(labels=False, title=None)),
    y=alt.Y('variable:N', sort=split, axis=alt.Axis(labels=False, title=None)),
    color=alt.Color('color_key:N', legend=None),
    order=alt.Order('split:Q'),
)

layers = [
    text,
    all_split,
    alt.Chart().mark_rect(
        x=0,
        x2=alt.ExprRef(f'datum.level_0 * {indent} * 0.9'),
        yOffset=20,
        height=60,
        color="#f8f8f8"
    ),
    alt.Chart().mark_rect(
        x=0,
        x2=alt.ExprRef(f'datum.level_0 * {indent} * 0.1'),
        yOffset=20,
        height=60,
        color="#008080"
    )
]

selection = alt.selection_single(empty='none')
selection2 = alt.selection_single(empty='none')
rect_layers = None
for l in ('tick', 'excepted', 'fail'):
    layer = alt.Chart().mark_rect(
        xOffset=alt.ExprRef(f'datum.level_0 * {indent}'),
        x2Offset=alt.ExprRef(f'datum.level_0 * {indent}'),
        yOffset=20,
        height=30,
        cornerRadius=10,
        cursor='pointer' if l == 'tick' else 'auto'
    )
    if l == 'fail':
        layer = layer.transform_filter(
            ~alt.datum.final
        )
    layer = layer.encode(
        x=alt.X(f'{l}_norm:Q', axis=alt.Axis(labels=False)), # apparently for either l
        color=alt.condition(
            selection,
            alt.value('#eeeeee'),
            alt.value(color_map_one[l]) # , legend=None, scale=None)
        ),
        #order=alt.Order('split'),
    )
    if not rect_layers:
        rect_layers = layer
    else:
        rect_layers = rect_layers + layer
        
fail_layer = alt.Chart().mark_rect(
    xOffset=alt.ExprRef(f'datum.level_0 * {indent}'),
    x2Offset=alt.ExprRef(f'datum.level_0 * {indent}'),
    yOffset=20,
    height=30,
    cornerRadius=10,
    cursor='pointer'
).transform_filter(
    alt.datum.final
).encode(
    x=alt.X(f'fail_norm:Q', axis=alt.Axis(labels=False)), # apparently for either l
    color=alt.condition(
        selection2,
        alt.value('#eeeeee'),
        alt.value('#dd0000') # , legend=None, scale=None)
    ),
    #order=alt.Order('split'),
).add_selection(selection2)
rect_layers = rect_layers + fail_layer

text_layer = alt.Chart().mark_text(
    xOffset=alt.ExprRef(f'datum.level_0 * {indent} + 5'),
    align='left',
    dx=-200,
    fontSize=alt.ExprRef('(1 - 0.2 * datum.level_0) * 20'),
    yOffset=10,
).transform_calculate(
    joined_samples='join(datum.samples, ", ")'
).encode(
    text="joined_samples:N",
    opacity=alt.condition(selection, alt.value(0.9), alt.value(0.05))
)
stat_layer = alt.Chart().mark_text(
    xOffset=alt.ExprRef(f'datum.level_0 * {indent} + 5'),
    align='left',
    dx=-200,
    fontSize=alt.ExprRef('(1 - 0.2 * datum.level_0) * 20'),
    yOffset=10,
    color='white',
    text=alt.ExprRef('round(datum.tick * 100 / datum.total) + "% (" + datum.tick + ")"'),
).encode(
    opacity=alt.condition(selection, alt.value(0), alt.value(1))
)

fail_text_layer = alt.Chart().mark_text(
    xOffset=alt.ExprRef(f'datum.level_0 * {indent} + 20'),
    align='left',
    dx=-200,
    fontSize=15,
    yOffset=30,
    color='red',
    fontWeight='bold'
).transform_filter(
    alt.datum.final
).transform_calculate(
    joined_samples='"..." + join(datum.samples_fail, ", ")'
).encode(
    text="joined_samples:N",
    opacity=alt.condition(selection, alt.value(0.9), alt.value(0.0))
)
exc_stat_layer = alt.Chart().mark_text(
    xOffset=alt.ExprRef(f'datum.level_0 * {indent} + 380'),
    align='right',
    dx=-200,
    fontSize=alt.ExprRef('(1 - 0.2 * datum.level_0) * 13'),
    yOffset=3,
    color='white',
    fontWeight='bold',
    text=alt.ExprRef(
        '"Exc: " + round(datum.excepted * 100 / datum.total) + "% (" + datum.excepted + ")"'
    ),
).transform_filter(
    alt.datum.final
).encode(
    opacity=alt.condition(selection, alt.value(0), alt.value(1))
)
fail_stat_layer = alt.Chart().mark_text(
    xOffset=alt.ExprRef(f'datum.level_0 * {indent} + 380'),
    align='right',
    dx=-200,
    fontSize=alt.ExprRef('(1 - 0.2 * datum.level_0) * 13'),
    yOffset=18,
    color='white',
    fontWeight='bold',
    text=alt.ExprRef(
        '"F: " + round(datum.fail * 100 / datum.total) + "% (" + datum.fail + ")"'
    ),
).transform_filter(
    alt.datum.final
).encode(
    opacity=alt.condition(selection, alt.value(0), alt.value(1))
)
layers.append(
    (rect_layers + text_layer + stat_layer + fail_text_layer + exc_stat_layer + fail_stat_layer).add_selection(selection)
)
#alt.themes.enable('vox')

a = alt.layer(
    *layers,
    data=exceptions_by_level
).transform_calculate(
    used="datum.pass + datum.fail"
).transform_calculate(
    excepted_norm="datum.excepted / datum.used",
    pass_norm="datum.pass / datum.used",
    tick_norm="datum.tick / datum.used",
    fail_norm="datum.fail / datum.used"
).facet(
    row=alt.Row('name', sort=alt.EncodingSortField('ix'), title=None, header=alt.Header(labels=False)),
    padding=0,
    spacing=0
).configure(
    font='League Spartan',
).configure_range(
    category=[
        "#008080", # excepted
        "#aaaaaa", # fail
        '#dd0000', # offset
        "#eeeeee", # pass
        "#444444"  # tick
    ],
).configure_view(strokeOpacity=0)
ad = a.to_dict()

In [505]:
ad['spec']['layer'][-1]['layer'][3]

{'mark': {'type': 'rect',
  'cornerRadius': 10,
  'cursor': 'pointer',
  'height': 30,
  'x2Offset': {'expr': 'datum.level_0 * 20'},
  'xOffset': {'expr': 'datum.level_0 * 20'},
  'yOffset': 20},
 'encoding': {'color': {'condition': {'value': '#eeeeee',
    'selection': 'selector211'},
   'value': '#dd0000'},
  'x': {'axis': {'labels': False},
   'field': 'fail_norm',
   'type': 'quantitative'}},
 'selection': {'selector211': {'type': 'single', 'empty': 'none'}},
 'transform': [{'filter': 'datum.final'}]}

In [506]:
layer = ad['spec']['layer'][-1]['layer']
layer[0]['encoding']['x2'] = {'field': 'tick_norm', 'type': 'quantitative'}
layer[0]['encoding']['x'] = {'datum': 0}
#layer[1]['selection'] = layer[0]['selection']
layer[1]['encoding']['x2'] = {'field': 'tick_norm', 'type': 'quantitative'}
layer[1]['encoding']['x'] = {'field': 'pass_norm', 'type': 'quantitative'}
#layer[2]['selection'] = {}
print(layer[0])
#layer[2]['selection'][layer[2]['encoding']['color']['condition']['selection']['or'][-1]] = layer[0]['selection'][layer[2]['encoding']['color']['condition']['selection']['or'][0]]
layer[2]['encoding']['x'] = {'field': 'pass_norm', 'type': 'quantitative'}
layer[2]['encoding']['x2'] = {'datum': 1}
layer[3]['encoding']['x'] = {'field': 'pass_norm', 'type': 'quantitative'}
layer[3]['encoding']['x2'] = {'datum': 1}
print(layer[2])

{'mark': {'type': 'rect', 'cornerRadius': 10, 'cursor': 'pointer', 'height': 30, 'x2Offset': {'expr': 'datum.level_0 * 20'}, 'xOffset': {'expr': 'datum.level_0 * 20'}, 'yOffset': 20}, 'encoding': {'color': {'condition': {'value': '#eeeeee', 'selection': 'selector210'}, 'value': '#27a827'}, 'x': {'datum': 0}, 'x2': {'field': 'tick_norm', 'type': 'quantitative'}}, 'selection': {'selector210': {'type': 'single', 'empty': 'none'}}}
{'mark': {'type': 'rect', 'cornerRadius': 10, 'cursor': 'auto', 'height': 30, 'x2Offset': {'expr': 'datum.level_0 * 20'}, 'xOffset': {'expr': 'datum.level_0 * 20'}, 'yOffset': 20}, 'encoding': {'color': {'condition': {'value': '#eeeeee', 'selection': 'selector210'}, 'value': '#008080'}, 'x': {'field': 'pass_norm', 'type': 'quantitative'}, 'x2': {'datum': 1}}, 'transform': [{'filter': '(!datum.final)'}]}


In [507]:
ad

{'config': {'view': {'continuousWidth': 400,
   'continuousHeight': 300,
   'strokeOpacity': 0},
  'font': 'League Spartan',
  'range': {'category': ['#008080',
    '#aaaaaa',
    '#dd0000',
    '#eeeeee',
    '#444444']}},
 'data': {'name': 'data-5022a735f9a1164d27174259071ee7b4'},
 'facet': {'row': {'field': 'name',
   'header': {'labels': False},
   'sort': {'field': 'ix'},
   'title': None,
   'type': 'nominal'}},
 'spec': {'layer': [{'mark': {'type': 'text',
     'align': 'left',
     'dx': -200,
     'fontSize': {'expr': '(1 - 0.2 * datum.level_0) * 20'},
     'xOffset': {'expr': 'datum.level_0 * 20'},
     'yOffset': -20},
    'encoding': {'text': {'field': 'name', 'type': 'nominal'}}},
   {'mark': {'type': 'bar',
     'height': 6,
     'xOffset': {'expr': 'datum.level_0 * 20'},
     'yOffset': -10},
    'encoding': {'color': {'field': 'color_key',
      'legend': None,
      'type': 'nominal'},
     'order': {'field': 'split', 'type': 'quantitative'},
     'x': {'axis': {'label

In [502]:
from altair import vegalite

vegalite.display.VegaLite._validate = lambda self: True
vegalite.display.VegaLite(ad)

In [508]:
import json
print(json.dumps(ad, indent=2))

{
  "config": {
    "view": {
      "continuousWidth": 400,
      "continuousHeight": 300,
      "strokeOpacity": 0
    },
    "font": "League Spartan",
    "range": {
      "category": [
        "#008080",
        "#aaaaaa",
        "#dd0000",
        "#eeeeee",
        "#444444"
      ]
    }
  },
  "data": {
    "name": "data-5022a735f9a1164d27174259071ee7b4"
  },
  "facet": {
    "row": {
      "field": "name",
      "header": {
        "labels": false
      },
      "sort": {
        "field": "ix"
      },
      "title": null,
      "type": "nominal"
    }
  },
  "spec": {
    "layer": [
      {
        "mark": {
          "type": "text",
          "align": "left",
          "dx": -200,
          "fontSize": {
            "expr": "(1 - 0.2 * datum.level_0) * 20"
          },
          "xOffset": {
            "expr": "datum.level_0 * 20"
          },
          "yOffset": -20
        },
        "encoding": {
          "text": {
            "field": "name",
            "type": "nomi